In [1]:
source("../Utility/doublets.r")
source("../Utility/seurat_tools.r")

In [2]:
library(parallel)
library(ggplot2)
library(qs)

qs 0.27.2. Announcement: https://github.com/qsbase/qs/issues/103



### My custom doublet finder function

In [3]:
View(callDoubletFinder)

function (seuratObj, min_pc, doublet_rate = 0.075, pN = 0.25, 
    res = 0.05, n_cores = 1) 
{
    seuratObj <- NormalizeData(seuratObj)
    seuratObj <- FindVariableFeatures(seuratObj)
    seuratObj <- ScaleData(seuratObj)
    seuratObj <- RunPCA(seuratObj, npcs = min_pc, features = VariableFeatures(seuratObj))
    seuratObj <- RunUMAP(seuratObj, dims = 1:min_pc)
    seuratObj <- FindNeighbors(seuratObj, dims = 1:min_pc)
    seuratObj <- FindClusters(seuratObj, resolution = res)
    sweep.list <- paramSweep(seuratObj, PCs = 1:min_pc, num.cores = n_cores)
    sweep.stats <- summarizeSweep(sweep.list)
    bcmvn <- find.pK(sweep.stats)
    bcmvn_max <- bcmvn[which.max(bcmvn$BCmetric), ]
    optimal_pk <- as.numeric(as.character(bcmvn_max$pK))
    annotations <- seuratObj@meta.data$seurat_clusters
    homotypic_prop <- modelHomotypic(annotations)
    nExp_poi <- round(doublet_rate * nrow(seuratObj@meta.data))
    nExp_poi_adj <- max(0, round(nExp_poi * (1 - homotypic_prop)))
    seuratObj <- doubletFinder(seu = seuratObj, PCs = 1:min_pc, 
        pN = pN, pK = optimal_pk, nExp = nExp_poi_adj)
    seuratObj@meta.data <- seuratObj@meta.data %>% rename_with(~paste0("DFC_", 
        seq_along(.)), matches("^DF.classifications")) %>% rename_with(~paste0("pANN_", 
        seq_along(.)), matches("^pANN_"))
    return(seuratObj)
}

### Load all samples count data

In [5]:
seurat_list <- qread("../Step2_Generate_Seurat_Object/out/seurat_list_SCPCP000001.qs")

### Apply doublet finder on each sample

In [ ]:
sink("./out/log_file_doubletFinder.txt", append = FALSE)
seurat_list_doublets <- list()
for (sample_name in names(seurat_list)) {
    clean_seurat_object <- clean_seurat(seurat_list[[sample_name]])
    seurat_list[[sample_name]] <- suppressMessages(
                                                suppressWarnings(
                                                    callDoubletFinder(
                                                                clean_seurat_object, 
                                                                min_pc=30, 
                                                                doublet_rate=0.075, 
                                                                pN=0.25, 
                                                                res=0.5, 
                                                                n_cores=20
                                                    )
                                            )
    )
} 
sink()

### Save output for downstream analysis

In [7]:
qsave(seurat_list, file = "./out/seurat_list_doublets_SCPCP000001.qs")

In [8]:
names(seurat_list)

[1] "SCPCL000002" "SCPCL000003" "SCPCL000004" "SCPCL000006" "SCPCL000007"
 [6] "SCPCL000008" "SCPCL000009" "SCPCL000010" "SCPCL000011" "SCPCL000013"
[11] "SCPCL000014" "SCPCL000015" "SCPCL000017" "SCPCL000018" "SCPCL000019"
[16] "SCPCL000020" "SCPCL000021" "SCPCL000022" "SCPCL000023"